In [1]:
from database import database 
from gmaps_places import gmaps_places
import json
import geopandas as gpd

#Read the API key (this is stored locally and should not be shared!)
with open('../keys/API_key_places.txt') as f:
    key = f.readlines()[0]

# Read the map of BCN
geo_bcn = gpd.read_file('bcn-open-data/geo/0301100100_UNITATS_ADM_POLIGONS.json')
border = geo_bcn.loc[0].geometry

# Create the gmap_places instance
places = gmaps_places(key)

# Set up the database connection
# Read the config file
f = open('config_file.json')
config = json.load(f)
my_db = database(config['host_name'], config['user_name'], config['password'], config['db'])

MySQL Server connection successful
MySQL Database connection test successful


In [2]:
# Create the grid and search 
places.create_grid_coords(border, distance = 200)
search = places.nearby_search(radius = 200)

#Insert the search results in the database
table = "restaurants"
for s in search:
    my_db.insert_query(table, s)

# Print and save a map of BCN with the restaurants found

In [ ]:
# BCN municipality border
bcn = gpd.GeoSeries(border)

# Grid points used for the search
bcn_grid = gpd.GeoSeries(grid_points)

# Read the coordinates from the database
restaurants_loc = my_db.execute_select_query("""SELECT loc_x, loc_y FROM restaurant;""")
bcn_rests = gpd.GeoSeries([Point(coord) for coord in restaurants_loc])

fig, ax = plt.subplots(1, 1, figsize = (4,4))
bcn.boundary.plot(ax = ax, color = 'k', label = 'Municipality border')
#bcn_grid.plot(markersize = 0.5, ax = ax)
bcn_rests.plot(markersize = 0.5, ax = ax, color = 'r', label = 'Restaurants')
fig.legend(loc = 'upper left', frameon = False)
ax.set_axis_off()
fig.savefig('Map_restaurants_BCN.pdf')
fig.savefig('Map_restaurants_BCN.png')